In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


# Introduction
Modern aircraft engines are equipped with hundreds of sensors monitoring temperature, pressure, vibration, and other critical parameters. Predicting Remaining Useful Life (RUL) is essential for:
* Preventing catastrophic engine failures
* Optimizing maintenance schedules
* Reducing downtime and operational costs
  
This notebook demonstrates a hybrid CNN + Transformer architecture to predict RUL from engine sensor time-series data.\
Advantages of this approach over traditional models like LSTM/GRU:
* Transformer layer: Captures long-range dependencies across cycles, which LSTMs may struggle with for long sequences.
* Asymmetric loss function: Penalizes overestimation of RUL (critical for safety-critical applications).
* Sensor-to-subsystem mapping: Helps interpret model decisions and provide actionable maintenance recommendations.

# Load & Preprocess (train / test)
We use the CMAPSS (Commercial Modular Aero-Propulsion System Simulation) dataset:\
Features:
* 3 operating settings (op_setting_1,2,3)
* 21 sensors monitoring engine subsystems such as Fan, LPC, HPC, Combustion Chamber, HPT
* Engine cycles: Sequential time steps for each engine

In [ ]:
# 1. Paths and sensor->subsystem mapping
TRAIN_PATH = '/kaggle/input/cmapss-jet-engine-simulated-data/train_FD001.txt'
TEST_PATH  = '/kaggle/input/cmapss-jet-engine-simulated-data/test_FD001.txt'
TRUTH_PATH = '/kaggle/input/cmapss-jet-engine-simulated-data/RUL_FD001.txt'

# Sensor to subsystem mapping provided by user
SENSOR_TO_SUBSYSTEM = {
    'sensor_1': ['Fan inlet temperature', 'Overall Inlet'],
    'sensor_2': ['Fan inlet pressure (psia)', 'Overall Inlet'],
    'sensor_3': ['HPC outlet pressure (psia)','High-Pressure Compressor (HPC)'],
    'sensor_4': ['HPT outlet temperature','High-Pressure Turbine (HPT)'],
    'sensor_5': ['LPC outlet temperature','Low-Pressure Compressor (LPC)'],
    'sensor_6': ['Fan speed rpm','Fan'],
    'sensor_7': ['Fan inlet pressure (psia)', 'Overall Inlet'],
    'sensor_8': ['HPC outlet static pressure (psia)','High-Pressure Compressor (HPC)'],
    'sensor_9': ['Corrected fan speed (rpm)','Fan'],
    'sensor_10': ['Ratio of pressure (bypass)Fan', 'Overall'],
    'sensor_11': ['Fuel flow (pph)','Combustion Chamber'],
    'sensor_12': ['LPC outlet temperature (corrected)','Low-Pressure Compressor (LPC)'],
    'sensor_13': ['HPT outlet temperature (corrected)','High-Pressure Turbine (HPT)'],
    'sensor_14': ['Physical fan speed (rpm)','Fan'],
    'sensor_15': ['Physical core engine speed (rpm)','High-Pressure Compressor (HPC) and High-Pressure Turbine (HPT)'],
    'sensor_16': ['Bleed air (units)','Overall Engine System'],
    'sensor_17': ['Fuel-air ratio','Combustion Chamber'],
    'sensor_18': ['Thrust-specific fuel consumption (TSFC)','Overall Engine System'],
    'sensor_19': ['Total temperature at LPC exit','Low-Pressure Compressor (LPC)'],
    'sensor_20': ['Engine pressure ratio','Overall Engine System'],
    'sensor_21': ['Total temperature at HPT exit ','High-Pressure Turbine (HPT)']
}

# 2. Load & preprocess data (features scaled; RUL kept in cycles)
train = pd.read_csv(TRAIN_PATH, sep=' ', header=None)
train.drop([26,27], axis=1, inplace=True)
cols = ['engine_id','cycle'] + [f'op_setting_{i+1}' for i in range(3)] + [f'sensor_{i+1}' for i in range(21)]
train.columns = cols

rul_df = train.groupby('engine_id')['cycle'].max().reset_index()
rul_df.columns = ['engine_id', 'max_cycle']
train = train.merge(rul_df, on='engine_id')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop('max_cycle', axis=1, inplace=True)

MAX_RUL = 125
train['RUL'] = train['RUL'].clip(upper=MAX_RUL)

feature_cols = train.columns[2:-1]
scaler = MinMaxScaler()
train[feature_cols] = scaler.fit_transform(train[feature_cols])

# test & truth
test = pd.read_csv(TEST_PATH, sep=' ', header=None)
test.drop([26,27], axis=1, inplace=True)
test.columns = cols

truth = pd.read_csv(TRUTH_PATH, header=None)
truth.columns = ['RUL']

# scale test
test[feature_cols] = scaler.transform(test[feature_cols])


# Sensor trends for all sensors with subsystem color coding
Visualize engine health trends for individual sensors mapped to subsystems:


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.0)

# Select a sample engine
sample_engine_id = train['engine_id'].unique()[0]
sample_engine_data = train[train['engine_id'] == sample_engine_id]

# Prepare color map for subsystems
subsystems = list(set([sub for subs in SENSOR_TO_SUBSYSTEM.values() for sub in subs]))
colors = sns.color_palette("tab20", len(subsystems))
subsystem_color_map = {sub: colors[i] for i, sub in enumerate(subsystems)}

# Plot each sensor in a grid
num_sensors = len(feature_cols)
cols = 3
rows = (num_sensors + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(15, rows*3), sharex=True)
axes = axes.flatten()

for i, sensor in enumerate(feature_cols):
    subs = SENSOR_TO_SUBSYSTEM.get(sensor, ['Unknown'])
    subsystem_color_map['Unknown'] = 'grey'
    color = subsystem_color_map[subs[0]]  # take first subsystem for color
    axes[i].plot(sample_engine_data['cycle'], sample_engine_data[sensor], color=color)
    axes[i].set_title(f"{sensor} ({subs[0]})", fontsize=10)
    axes[i].set_ylabel("Normalized Value")
    axes[i].grid(True)

# Remove empty subplots if any
for j in range(num_sensors, len(axes)):
    fig.delaxes(axes[j])

plt.xlabel("Cycle")
plt.suptitle(f"Sensor Trends over Cycles for Engine {sample_engine_id}", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


## Sequence Builder (train and test)

In [ ]:
# 3. Sequence creation helpers
SEQ_LEN = 30

def create_sequences(df, seq_length=SEQ_LEN):
    X, y = [], []
    for eid in df['engine_id'].unique():
        ed = df[df['engine_id']==eid].reset_index(drop=True)
        for i in range(len(ed)-seq_length):
            X.append(ed.iloc[i:i+seq_length][feature_cols].values)
            y.append(ed.iloc[i+seq_length]['RUL'])
    return np.array(X), np.array(y)

def create_sequences_test_last(df, seq_length=SEQ_LEN):
    X, engine_ids = [], []
    for eid in df['engine_id'].unique():
        ed = df[df['engine_id']==eid].reset_index(drop=True)
        if len(ed) >= seq_length:
            X.append(ed.iloc[-seq_length:][feature_cols].values)
            engine_ids.append(eid)
    return np.array(X), np.array(engine_ids)

X, y = create_sequences(train, SEQ_LEN)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, test_engine_ids = create_sequences_test_last(test, SEQ_LEN)
true_rul_test = truth['RUL'].values[:len(test_engine_ids)]
print('Shapes -', X_train.shape, X_val.shape, X_test.shape)



## Dataset & Dataloader

In [ ]:
# 4. Dataset & dataloader
class RUL_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1,1)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

BATCH = 64
train_loader = DataLoader(RUL_Dataset(X_train, y_train), batch_size=BATCH, shuffle=True)
val_loader = DataLoader(RUL_Dataset(X_val, y_val), batch_size=BATCH, shuffle=False)


## Model definition (parameterize cnn_channels)

In [ ]:
# 5. Model definition (parameterize cnn_channels)
class CNNTransformerModel(nn.Module):
    def __init__(self, input_dim, cnn_channels=64, d_model=128, nhead=4, num_layers=2, dropout=0.1):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(input_dim, cnn_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(cnn_channels, cnn_channels, kernel_size=3, padding=1),
            nn.ReLU()
        )
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.input_proj = nn.Linear(cnn_channels, d_model)
        self.fc = nn.Sequential(nn.Linear(d_model,64), nn.ReLU(), nn.Dropout(0.2), nn.Linear(64,1))

    def forward(self, x):
        x = x.permute(0,2,1)  # [B, F, T]
        x = self.cnn(x)       # [B, C, T]
        x = x.permute(0,2,1)  # [B, T, C]
        x = self.input_proj(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.fc(x)


Advantages over LSTM/GRU:
* CNN captures short-term temporal correlations efficiently.
* Transformer captures long-term dependencies without vanishing gradients.
* Parallelizable for faster training on GPUs.
* Better feature extraction from multisensor sequences.

## Asymmetric loss and metrics

In [ ]:
# 6. Asymmetric loss and metrics
mse_loss = nn.MSELoss()

def asymmetric_loss(pred, true, lambda_over=10.0):
    base = mse_loss(pred, true)
    over_pen = torch.relu(pred - true)
    over_term = (over_pen**2).mean()
    return base + lambda_over * over_term

def compute_overestimation_rate(preds, trues):
    over = (preds - trues) > 0
    return float(over.sum()) / len(over)

# Modified asymmetric loss to weight by true RUL
def weighted_asymmetric_loss(pred, true, lambda_over=10.0, max_rul=125):
    """
    - Penalizes overestimation (lambda_over)
    - Gives higher weight to high RUL samples
    """
    base = mse_loss(pred, true)
    over_pen = torch.relu(pred - true)
    over_term = (over_pen**2).mean()
    
    # Weight by true RUL (normalized)
    weight = (true / max_rul).view(-1,1)  # shape [batch_size,1]
    weighted_base = (weight * (pred - true)**2).mean()
    
    return weighted_base + lambda_over * over_term

* Penalizes overestimation (critical for safety)
* Can also weight loss by true RUL for more focus on long-lived engines

# Training & evaluation functions
* Split training sequences into train/validation sets
* Use Adam optimizer and early stopping
* Hyperparameter tuning: learning rate, CNN channels, overestimation penalty

In [ ]:
# 7. Training & evaluation functions

def train_one_epoch(model, loader, optimizer, lambda_over):
    model.train()
    total = 0.0
    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(Xb)
        loss = asymmetric_loss(preds, yb, lambda_over=lambda_over)
        loss.backward()
        optimizer.step()
        total += loss.item() * Xb.size(0)
    return total / len(loader.dataset)

def train_one_epoch_weighted(model, loader, optimizer, lambda_over, max_rul=125):
    model.train()
    total = 0.0
    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(Xb)
        loss = weighted_asymmetric_loss(preds, yb, lambda_over=lambda_over, max_rul=max_rul)
        loss.backward()
        optimizer.step()
        total += loss.item() * Xb.size(0)
    return total / len(loader.dataset)


def evaluate(model, loader):
    model.eval()
    all_preds, all_trues = [], []
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            preds = model(Xb)
            all_preds.append(preds.cpu().numpy().flatten())
            all_trues.append(yb.cpu().numpy().flatten())
    all_preds = np.concatenate(all_preds)
    all_trues = np.concatenate(all_trues)
    val_mse = mean_squared_error(all_trues, all_preds)
    over_rate = compute_overestimation_rate(all_preds, all_trues)
    return val_mse, over_rate, all_preds, all_trues


## Hyperparameter grid search

In [ ]:
# 8. Hyperparameter grid search
search_space = {
    'lr': [1e-3, 5e-4],
    'lambda_over': [5.0, 10.0, 20.0],
    'cnn_channels': [32, 64]
}

BEST_DIR = 'hp_search_results'
os.makedirs(BEST_DIR, exist_ok=True)

results = []

for lr in search_space['lr']:
    for lambda_over in search_space['lambda_over']:
        for cnn_ch in search_space['cnn_channels']:
            print(f"Running grid point lr={lr}, lambda_over={lambda_over}, cnn_ch={cnn_ch}")
            model = CNNTransformerModel(input_dim=len(feature_cols), cnn_channels=cnn_ch).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            EPOCHS = 25
            best_val_obj = float('inf')
            patience = 6
            no_improve = 0
            for epoch in range(EPOCHS):
                train_obj = train_one_epoch_weighted(model, train_loader, optimizer, lambda_over)
                val_mse, val_over, _, _ = evaluate(model, val_loader)
                # objective combines MSE and over-rate penalty
                val_obj = val_mse + 1000.0 * val_over
                print(f"  Epoch {epoch+1}/{EPOCHS} train_obj={train_obj:.4f} val_mse={val_mse:.4f} val_over={val_over:.3f} val_obj={val_obj:.4f}")
                if val_obj < best_val_obj:
                    best_val_obj = val_obj
                    torch.save(model.state_dict(), os.path.join(BEST_DIR, f"best_lr{lr}_lam{lambda_over}_ch{cnn_ch}.pth"))
                    no_improve = 0
                else:
                    no_improve += 1
                    if no_improve >= patience:
                        print("  Early stopping")
                        break
            # after training, load best and evaluate
            model.load_state_dict(torch.load(os.path.join(BEST_DIR, f"best_lr{lr}_lam{lambda_over}_ch{cnn_ch}.pth"), map_location=device))
            val_mse, val_over, val_preds, val_trues = evaluate(model, val_loader)
            results.append({'lr': lr, 'lambda_over': lambda_over, 'cnn_ch': cnn_ch,
                            'val_mse': val_mse, 'val_over': val_over, 'val_obj': val_mse + 1000.0*val_over})
            # save summary CSV per grid point
            pd.DataFrame(results).to_csv(os.path.join(BEST_DIR, 'grid_search_summary.csv'), index=False)

# print sorted results
res_df = pd.DataFrame(results).sort_values('val_obj')
print('Top grid results:')
print(res_df.head())


## Load best hyperparameters & final evaluation on test
* Metrics:
    * RMSE: Measures prediction accuracy
    * Overestimation rate: Fraction of predictions over true RUL

In [ ]:
# 9. Load best hyperparameters & final evaluation on test
best = res_df.iloc[0]
print('Selected best config:', best.to_dict())
best_model_path = os.path.join(BEST_DIR, f"best_lr{best['lr']}_lam{best['lambda_over']}_ch{int(best['cnn_ch'])}.pth")
model = CNNTransformerModel(input_dim=len(feature_cols), cnn_channels=int(best['cnn_ch'])).to(device)
model.load_state_dict(torch.load(best_model_path, map_location=device))

# test predictions
model.eval()
with torch.no_grad():
    X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_pred_test = model(X_test_t).cpu().numpy().flatten()

rmse_test = math.sqrt(mean_squared_error(true_rul_test, y_pred_test))
over_test = compute_overestimation_rate(y_pred_test, true_rul_test)
print(f"Final Test RMSE: {rmse_test:.3f}  Overestimation rate: {over_test*100:.2f}%")

results_df = pd.DataFrame({'engine_id': test_engine_ids, 'true_RUL': true_rul_test, 'predicted_RUL': y_pred_test})
results_df['error'] = results_df['predicted_RUL'] - results_df['true_RUL']
results_df.to_csv('final_predictions.csv', index=False)


## Attribution (SmoothGrad Grad x Input) and mapping to subsystems

In [ ]:
# 10. Attribution (SmoothGrad Grad x Input) and mapping to subsystems

def compute_grad_times_input_smooth_single(model, x_np, n_samples=12, stdev=1e-3):
    model.eval()
    X = torch.tensor(x_np, dtype=torch.float32).unsqueeze(0).to(device)
    accum = np.zeros_like(x_np)
    for _ in range(n_samples):
        noise = torch.randn_like(X) * stdev
        Xn = (X + noise).clone().detach().requires_grad_(True)
        out = model(Xn)
        out.sum().backward()
        grads = Xn.grad.detach().cpu().numpy()[0]
        inputs = Xn.detach().cpu().numpy()[0]
        accum += np.abs(grads * inputs)
    return accum / n_samples

reports = []
THRESHOLD_RUL_FLAG = 30
for i, (eid, pred, true_rul) in enumerate(zip(test_engine_ids, y_pred_test, true_rul_test)):
    attr_map = compute_grad_times_input_smooth_single(model, X_test[i])  # [seq_len, features]
    feat_imp = attr_map.sum(axis=0)
    feat_imp_norm = feat_imp / (feat_imp.sum() + 1e-12)
    top_idx = feat_imp_norm.argsort()[::-1][:5]
    top_sensors = [(feature_cols[idx], float(feat_imp_norm[idx])) for idx in top_idx]
    # map sensors to subsystems and aggregate
    subsystem_scores = {}
    for idx in range(len(feature_cols)):
        sname = feature_cols[idx]
        sensor_key = sname  # e.g., 'sensor_1'
        subs = SENSOR_TO_SUBSYSTEM.get(sensor_key, ['Unknown'])
        for sub in subs:
            subsystem_scores[sub] = subsystem_scores.get(sub, 0.0) + float(feat_imp_norm[idx])
    # sort subsystems
    subs_sorted = sorted(subsystem_scores.items(), key=lambda x: x[1], reverse=True)
    imminent = bool(pred <= THRESHOLD_RUL_FLAG)
    reports.append({
        'engine_id': int(eid),
        'predicted_RUL': float(pred),
        'true_RUL': int(true_rul),
        'imminent_failure_flag': imminent,
        'top_sensors': top_sensors,
        'top_subsystems': subs_sorted[:5]
    })

reports_df = pd.DataFrame(reports)
reports_df.to_csv('engine_failure_reports.csv', index=False)

In [ ]:
reports_df.head()

## Results Visualization
* Predicted vs True RUL Scatter
* Error distribution
* Top contributing sensors/subsystems

In [ ]:
# === 12. Visualization for LinkedIn / Reports ===

import seaborn as sns
sns.set(style="whitegrid", font_scale=1.1)

# 12.1 True vs Predicted RUL for test engines
plt.figure(figsize=(10,6))
plt.scatter(results_df['true_RUL'], results_df['predicted_RUL'], c='blue', alpha=0.6)
plt.plot([0, MAX_RUL],[0, MAX_RUL], 'r--', label='Ideal')
plt.xlabel("True RUL (cycles)")
plt.ylabel("Predicted RUL (cycles)")
plt.title("Predicted vs True RUL for Test Engines")
plt.legend()
plt.tight_layout()
plt.savefig("pred_vs_true_rul.png", dpi=150)
plt.show()


# 12.2 Error distribution across test engines
plt.figure(figsize=(10,5))
sns.histplot(results_df['error'], bins=20, kde=True, color='orange')
plt.axvline(0, color='black', linestyle='--')
plt.xlabel("Prediction Error (Predicted - True RUL)")
plt.ylabel("Count")
plt.title("Distribution of RUL Prediction Errors")
plt.tight_layout()
plt.savefig("rul_error_distribution.png", dpi=150)
plt.show()


# 12.3 Feature importance (averaged across all engines)
avg_feat_imp = np.zeros(len(feature_cols))
for i in range(len(X_test)):
    attr_map = compute_grad_times_input_smooth_single(model, X_test[i])
    avg_feat_imp += attr_map.sum(axis=0)
avg_feat_imp /= len(X_test)
avg_feat_imp_norm = avg_feat_imp / (avg_feat_imp.sum() + 1e-12)

feat_imp_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': avg_feat_imp_norm
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12,6))
sns.barplot(data=feat_imp_df, x='feature', y='importance', palette="viridis")
plt.xticks(rotation=45, ha='right')
plt.title("Feature Importance (SmoothGrad x Input, averaged over test engines)")
plt.tight_layout()
plt.savefig("feature_importance.png", dpi=150)
plt.show()


# 12.4 Subsystem importance (aggregated over engines)
subsystem_scores = {}
for i in range(len(X_test)):
    attr_map = compute_grad_times_input_smooth_single(model, X_test[i])
    feat_imp = attr_map.sum(axis=0)
    feat_imp_norm = feat_imp / (feat_imp.sum() + 1e-12)
    for idx, sensor_key in enumerate(feature_cols):
        subs = SENSOR_TO_SUBSYSTEM.get(sensor_key, ['Unknown'])
        for sub in subs:
            subsystem_scores[sub] = subsystem_scores.get(sub, 0) + feat_imp_norm[idx]

subsystem_df = pd.DataFrame(list(subsystem_scores.items()), columns=['subsystem','importance']).sort_values('importance', ascending=False)

plt.figure(figsize=(10,5))
sns.barplot(data=subsystem_df, x='subsystem', y='importance', palette='magma')
plt.xticks(rotation=45, ha='right')
plt.title("Subsystem Importance (Aggregated from sensor attributions)")
plt.tight_layout()
plt.savefig("subsystem_importance.png", dpi=150)
plt.show()

# Line Graph: Predicted vs True RUL ===

import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(test_engine_ids, true_rul_test, label='True RUL', marker='o', linestyle='-', color='blue')
plt.plot(test_engine_ids, y_pred_test, label='Predicted RUL', marker='x', linestyle='--', color='red')
plt.xlabel("Engine ID")
plt.ylabel("Remaining Useful Life (cycles)")
plt.title("Predicted vs True RUL for Test Engines")
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("linegraph_pred_vs_true_rul.png", dpi=150)
plt.show()

## Generate templated maintenance reports for imminent engines
* Highlight top sensors indicating degradation
* Identify subsystems likely to fail
* Provide inspection recommendations based on predicted RUL\
Optional: Generate professional reports using a local LLM

In [ ]:
# 11. Generate templated maintenance reports for imminent engines (save all, print one)

def generate_template_report(entry):
    eid = entry['engine_id']
    pred = entry['predicted_RUL']
    true_rul = entry['true_RUL']
    sensors = entry['top_sensors']
    subs = entry['top_subsystems']
    s_text = ', '.join([f"{name} ({weight:.2f})" for name, weight in sensors])
    subs_text = ', '.join([f"{name} ({weight:.2f})" for name, weight in subs])
    lines = [
        f"Engine ID: {eid}",
        f"Predicted remaining life: {pred:.1f} cycles (true RUL: {true_rul})",
        f"Top sensors indicating degradation: {s_text}",
        f"Likely affected subsystems: {subs_text}",
        "",
        "Maintenance recommendation:",
        "- Inspect subsystems listed above and their sensor harnesses/fixtures.",
        "- For temperature sensors: check for thermocouple drift, insulation, fuel/air mixing anomalies.",
        "- For pressure sensors: check for leaks, blockages, or compressor stalls.",
        "- For vibration/rotational sensors: inspect bearings, couplings, and mounts.",
        "- If sensor replacement is performed, recalibrate and run diagnostic cycles; re-evaluate RUL.",
        "- Consider immediate grounding if predicted RUL <= 10 cycles until inspection completes."
    ]
    return "\n".join(lines)

imminent = [r for r in reports if r['imminent_failure_flag']]

if imminent:
    # Print the first imminent report
    print("===== Maintenance Report (First Imminent Engine) =====")
    print(generate_template_report(imminent[0]))
    
    # Save all imminent engine reports to files
    for r in imminent:
        txt = generate_template_report(r)
        fname = f"engine_{r['engine_id']}_maintenance_report.txt"
        with open(fname, 'w') as f:
            f.write(txt)
else:
    print("No imminent engines flagged for maintenance.")

print('Saved final outputs: final_predictions.csv, engine_failure_reports.csv, engine_*_maintenance_report.txt')


## Optional: LLM rephrasing using HuggingFace

In [ ]:
# 11. Generate maintenance reports using a local LLM (from Kaggle input)
'''
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Path to the model in Kaggle input
model_name = "/kaggle/input/qwen2.5/transformers/0.5b/1"

# Load tokenizer and model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate_llm_report(entry, max_new_tokens=300):
    eid = entry['engine_id']
    pred = entry['predicted_RUL']
    true_rul = entry['true_RUL']
    sensors = entry['top_sensors']
    subs = entry['top_subsystems']
    
    s_text = ', '.join([f"{name} ({weight:.2f})" for name, weight in sensors])
    subs_text = ', '.join([f"{name} ({weight:.2f})" for name, weight in subs])
    
    prompt = f"""
You are an aircraft maintenance expert. Generate a professional maintenance report for this engine:

Engine ID: {eid}
Predicted RUL: {pred:.1f} cycles
True RUL: {true_rul}
Top sensors indicating degradation: {s_text}
Likely affected subsystems: {subs_text}

Provide detailed recommendations, including inspections, sensor checks, subsystem replacements if needed, safety measures, and urgency based on RUL.
"""

    # Tokenize and generate text
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.2,
        do_sample=False
    )
    llm_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    report_lines = [
        f"Engine ID: {eid}",
        f"Predicted remaining life: {pred:.1f} cycles (true RUL: {true_rul})",
        f"Top sensors indicating degradation: {s_text}",
        f"Likely affected subsystems: {subs_text}",
        "",
        "Maintenance Recommendation (LLM-generated):",
        llm_text
    ]
    return "\n".join(report_lines)

# Generate reports for imminent failures
imminent = [r for r in reports if r['imminent_failure_flag']]
for r in imminent:
    txt = generate_llm_report(r)
    fname = f"engine_{r['engine_id']}_maintenance_llm_report.txt"
    with open(fname, 'w') as f:
        f.write(txt)

print('Saved final outputs: final_predictions.csv, engine_failure_reports.csv, engine_*_maintenance_report.txt')
'''

## Conclusion
* CNN-Transformer outperforms classical LSTM/GRU on long sequences.
* Asymmetric loss prevents overestimation, enhancing safety.
* SmoothGrad attribution provides actionable insights for maintenance teams.
* Combined with LLM-generated reports, this pipeline bridges predictive analytics and maintenance decision support.